In [1]:
import torch
from ModelAi.projectFeature import PJFTrainModel, PJF
import pandas as pd

PJF_model = PJF()
PJF_model.to("cuda")
model = PJFTrainModel(model=PJF_model)
model.to("cuda")

model.load_state_dict(
    torch.load(
        f"D:\\AIChallenge\\Checkpoints\\model_epoch_{0}.pt",
    )
)

PJF_model = model.model
PJF_model.eval()

from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm

# BARTpho-word
word_tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word-base")
bartpho_word = AutoModel.from_pretrained("vinai/bartpho-word-base")
# to cuda
bartpho_word.to("cuda")
bartpho_word.eval()
for param in bartpho_word.parameters():
    param.requires_grad = False

d:\AIChallenge\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1602: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
d:\AIChallenge\venv\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
# Word segmentation
import py_vncorenlp

# py_vncorenlp.download_model(save_dir=r"D:\VQA\Notebook\VnCoreNLP")

# Load the word and sentence segmentation component
rdrsegmenter = py_vncorenlp.VnCoreNLP(
    annotators=["wseg"],
    save_dir=r"D:\AIChallenge\VnCoreNLP",
)

# change dir to C:\Users\Asus.LAPTOP-8EU9PHJL.000\Desktop\VQA\Notebook
import os

os.chdir(r"D:\AIChallenge")


def Word_Segmentation(text):
    output = rdrsegmenter.word_segment(text)
    if len(output) > 0:
        # join all words in the list
        res = ""
        for i in output:
            res += i + " "
        return res
    else:
        output = ""
    return output


print(Word_Segmentation("Chúng tôi là những nghiên cứu viên."))

Chúng_tôi là những nghiên_cứu_viên . 


In [3]:
import os
from tqdm import tqdm
import torch


def scan_file_path_in_folder(directory):
    lst = []
    for root, dirs, files in os.walk(directory):
        # check files in the directory
        for file in files:
            lst.append(os.path.join(root, file))
    return lst


def scan_file_name_in_folder(directory):
    lst = []
    for root, dirs, files in os.walk(directory):
        # check files in the directory
        for file in files:
            # remove the extension
            file_name = os.path.splitext(file)[0]
            lst.append(file_name)
    return lst

In [4]:
# load json file
import json

with open("clean_data.json", "r") as f:
    cleanData = json.load(f)

In [5]:
import torch
import concurrent.futures
from tqdm import tqdm


def load_tensor(path):
    # load tensor if it exists else return a tensor of zeros of shape (1, 1024)
    res = torch.zeros(1, 1024)
    if os.path.exists(path):
        res = torch.load(path)
    if res.shape[0] == 0:
        res = torch.zeros(1, 1024)
    return res


def process_video(video_name, path):
    return load_tensor(path)


def process_list_tensor(video_name, path_lst):
    # stack all the tensors in the list
    return torch.stack([load_tensor(path) for path in path_lst])


with concurrent.futures.ThreadPoolExecutor() as executor:
    tensors = {
        executor.submit(process_video, video_name, val["path"]): video_name
        for video_name, val in cleanData.items()
    }
    audio_key_tensors = {
        executor.submit(
            process_list_tensor, video_name, val["audio_key_path_lst"]
        ): video_name
        for video_name, val in cleanData.items()
    }

    key_image_caption_tensors = {
        executor.submit(
            process_list_tensor, video_name, val["key_image_caption_path_lst"]
        ): video_name
        for video_name, val in cleanData.items()
    }

    BEITv2_tensors = {
        executor.submit(
            process_list_tensor, video_name, val["BEITv2_path_lst"]
        ): video_name
        for video_name, val in cleanData.items()
    }

    for future in tqdm(
        concurrent.futures.as_completed(tensors),
        total=len(tensors),
        desc="Loading video tensors",
    ):
        video_name = tensors[future]
        cleanData[video_name]["video_tensor"] = future.result()

    for future in tqdm(
        concurrent.futures.as_completed(audio_key_tensors),
        total=len(audio_key_tensors),
        desc="Loading audio key tensors",
    ):
        video_name = audio_key_tensors[future]
        cleanData[video_name]["audio_key_tensor"] = future.result()

    for future in tqdm(
        concurrent.futures.as_completed(key_image_caption_tensors),
        total=len(key_image_caption_tensors),
        desc="Loading key image caption tensors",
    ):
        video_name = key_image_caption_tensors[future]
        cleanData[video_name]["key_image_caption_tensor"] = future.result()

    for future in tqdm(
        concurrent.futures.as_completed(BEITv2_tensors),
        total=len(BEITv2_tensors),
        desc="Loading BEITv2 tensors",
    ):
        video_name = BEITv2_tensors[future]
        cleanData[video_name]["BEITv2_tensor"] = future.result()

# stack all the video tensors of cleanData
cleanData["video_tensors"] = torch.stack(
    [val["video_tensor"] for val in cleanData.values()]
)

Loading audio key tensors:  90%|████████▉ | 652/726 [01:02<00:07, 10.45it/s]


RuntimeError: stack expects a non-empty TensorList

In [ ]:
# query_path = "pack1-groupA"
query_path = "pack2-groupA"
query_path_lst = scan_file_path_in_folder(query_path)
query_path_name_lst = scan_file_name_in_folder(query_path)
print(query_path_lst)

In [ ]:
for i in tqdm(len(query_path)):
    # read the text
    with open(query_path_lst[i], "r", encoding="utf8") as f:
        text = f.read()